In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import glob, functools, tqdm, PIL
from multiprocess import Pool
from PIL import Image
import io
import pathlib

im = Image.open('apple.jpg', 'r')
im2 = Image.open('apple2.jpg', 'r')
#im = Image.open({'path':glob.glob("../training/Motorola-X/(MotoX)1.jpg")})
pix = im.load()
print (im.size)
matrix = np.array(im.getdata())#convert the images as a set of pixels
print(matrix)

df = pd.DataFrame(data=matrix) #convert numpy array type to a df

im2.rotate(45).save('apple3.png') #rotating a photo omg!

Attempt 1 Aggregate Data

In [54]:
training = []
#"C:\Users\Nikkitha\Documents\COMPUTER ENGR\CMPE 188\project\train\HTC-1-M7\(HTC-1-M7)1.jpg"
for infile in glob.glob( "..\train\HTC-1-M7\*.jpg"):
    im = Image.open(infile)
    matrix = np.array(im.getdata())
    array = np.concatenate(training, matrix)

print(array)

FileNotFoundError: [Errno 2] No such file or directory: 'path'

Attempt 2 Aggregate Data

In [ ]:
import cv2
import os
import glob
img_dir = "train/Motorola-X" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)
    
data

Attempt 3 Aggregate Data

In [50]:
import cv2
import os
from skimage.io import imread_collection, imshow_collection, concatenate_images
import numpy as np

folder = 'apples/*.jpg'

col = imread_collection(folder) #Concatenate all images in the image collection into an array.

n = 2
#array = np.empty(shape=[1,n])
#imshow_collection(col)
w,h,l = 1,2,2
array = [[[0 for x in range(w)] for y in range(h)] for z in range(l)]
array = concatenate_images(col)
  

ValueError: Image dimensions must agree.

# Below is the example code

In [ ]:

def imread(fn):
    return np.array(PIL.Image.open(fn))

def imsize(fn):
    im = PIL.Image.open(fn)
    sz = im.size
    im.close()
    return sz

def is_landscape(fn):
    s = imsize(fn)
    return s[0] > s[1]

In [15]:
train = pd.DataFrame({'path':glob.glob('../train/*/*')})
train['modelname'] = train.path.map(lambda p:p.split('/')[-2])

,path,modelname


In [16]:
import cv2

def random_crop_fft(img, W):
    nr, nc = img.shape[:2]
    r1, c1 = np.random.randint(nr-W), np.random.randint(nc-W) 
    imgc = img[r1:r1+W, c1:c1+W, :]

    img1 = imgc - cv2.GaussianBlur(imgc, (3,3), 0)
    imgs1 = np.sum(img1, axis=2)
    
    sf = np.stack([
         np.fft.fftshift(np.fft.fft2( imgs1 )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,0] - img1[:,:,1] )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,1] - img1[:,:,2] )),
         np.fft.fftshift(np.fft.fft2( img1[:,:,2] - img1[:,:,0] )) ], axis=-1)
    return np.abs(sf)
    
def imread_residual_fft(fn, W, navg):
    #print(fn, rss())
    img = imread(fn).astype(np.float32) / 255.0
    return sum(map(lambda x:random_crop_fft(img, W), range(navg))) / navg

def noise_pattern(modelname, W, navg=256):
    files = train.path[train.modelname == modelname].values
    orientations = np.vectorize(is_landscape)(files)
    if np.sum(orientations) < len(orientations)//2:
        orientations = ~orientations
    files = files[orientations]

    from multiprocess import Pool
    with Pool() as pool:
        s = sum(tqdm.tqdm(pool.imap(lambda fn:imread_residual_fft(fn, W, navg), files), total=len(files), desc=modelname)) / len(files)
    
    return s

In [17]:
def plot_model_features(modelname, W):
    s = noise_pattern(modelname, W)
    nchans = s.shape[2]
    nrows = (nchans + 3) // 4
    _, ax = plt.subplots(nrows, 4, figsize=(16, 4 * nrows))
    ax = ax.flatten()

    for c in range(nchans):
        eps = np.max(s[:,:,c]) * 1e-2
        s1 = np.log(s[:,:,c] + eps) - np.log(eps) 
        img = (s1 * 255 / np.max(s1)).astype(np.uint8)
        ax[c].imshow(cv2.equalizeHist(img))
        
    for ax1 in ax[nchans:]:
        ax1.axis('off')

    plt.show()
    
def plot_all_model_features(W):
    print("Feature Size={}".format(W))
    for modelname in train.modelname.unique():
        plot_model_features(modelname, W)


In [18]:
plot_all_model_features(W=128)

Feature Size=128
